In [211]:
import torch

In [212]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import butter, lfilter , filtfilt
import pandas as pd
import scipy as sp
import os
import sys
import seaborn as sb 
from matplotlib import cm

import pickle


# Load the Dic

In [213]:

class DfSubject:
    def __init__(self, elec_i, muscle_i):
        self.Elec_config=list_elecs[elec_i]
        self.List_elecs=list_elecs
        self.Muscle_or_other=list_muscle_or_other_elec_i[muscle_i]
        self.List_muscle_or_other=list_muscle_or_other_elec_i
        self.Trial_number= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Trial_number"]
        self.Trial_number=[int(i) for i in self.Trial_number] #save as int
        self.Raw= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Raw"]
        self.Raw.index=self.Trial_number
        self.Time_points= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Time_points"]
        self.Time_points.index=self.Trial_number
        self.Amplitudes= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Amplitudes"]
        self.Amplitudes.index=self.Trial_number
        self.TS_arts_1="$"
        
        # self.TS_arts_1.index=self.Trial_number
        self.Time_points_arts_1=-1
        self.TS_arts_2="$"
        self.Time_points_arts_2=-1
        self.TS_arts_3="$"
        self.Time_points_arts_3=-1


# Goal is to chose an electrode and a muscle and see all the trials and the corrected data        
#  Access data the following way: pData["_3_5"].Raw[19] #prints the raw data from trial 19

In [214]:
### retrieve  dfSubject structured as pData  #
#run the structure of the data before opening the pickle (wihtout creating an object)
# command pData["_0_0"].Trial_number=0
import pickle
run=1
if run == 1:  

    with open('Class2_Dump.pkl', 'rb') as handle:
        pData = pickle.load(handle)


# Make a tensor (include all data in one tensor)

Build the tensor matrix with the raw data.
Time X Electrodes X Muscles X Amplitudes (Stimulation) X Muscle activity (EMG)
Right now it's a table (16410(=all the trials) x 14440(TP+AMp+RAw(1404)+Elec(20)+Muscle(15)))
Since the data doesn't have the same dimension I wouldn't know how to stack? eg tensor.shape(5, 16410, ?)
->Once I have the specified nr. of columns I could adapt the different lists and use tensor.reshape(5,16410, ?)


In [215]:
device =  "cuda" if torch.cuda.is_available() else "cpu"

### Extract list

The 5 dimensions that should be included in the tensor 
    
    elecs_to_nr
    muscles_to_nr 
    raw_data_to_matrix, 
    amplitudes_to_list, 
    time_points_to_list

In [216]:

## save Amplitudes in a list ##
amplitudes_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                amplitudes_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial])
                

print(len(amplitudes_to_list)) #16410

16410


In [217]:

## save raw_data in a list ##
#with append() in order to save lists of lists
raw_data_to_matrix=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                raw_data_to_matrix.append(pData[f'_{elec_i}_{muscle_i}'].Raw[trial])
               

print(len(raw_data_to_matrix))
print((raw_data_to_matrix[0].dtype)) #float64
# print((raw_data_to_matrix[0:10].dtype)) #error


16410
float64


In [218]:

## save the Time_points in a list ##
time_points_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                time_points_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Time_points[trial])
                

print(len(time_points_to_list)) #16410


16410


In [219]:
## save Muscles in a list ##
muscles_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                muscles_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Muscle_or_other)
                

print(len(muscles_to_list)) #16410
print(muscles_to_list[0:10])

16410
['LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST']


In [220]:
## save Elecs in a list ##
elecs_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                elecs_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Elec_config)
                

print(len(elecs_to_list)) #16410
print(elecs_to_list[0:10])

16410
['Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_']


#### Use Hot encoder to convert from str to number
for muscles and elecs (since the data is str and for the tensor can only have one data type)

In [221]:
# Use hot encoder to convert elecs to nrs
import pandas as pd
elecs_to_nr=pd.get_dummies(
    np.array(elecs_to_list), 
    prefix=None, 
    prefix_sep='_', 
    dummy_na=False, 
    columns=None, 
    sparse=False, 
    drop_first=False, 
    dtype=None
)

In [222]:
# Use hot encoder to convert muscles to nrs
import pandas as pd
muscles_to_nr=pd.get_dummies(
    np.array(muscles_to_list), 
    prefix=None, 
    prefix_sep='_', 
    dummy_na=False, 
    columns=None, 
    sparse=False, 
    drop_first=False, 
    dtype=None
)

In [223]:
# Use hot encoder to convert elecs to nrs
import pandas as pd
elecs_to_nr=pd.get_dummies(
    np.array(elecs_to_list), 
    prefix=None, 
    prefix_sep='_', 
    dummy_na=False, 
    columns=None, 
    sparse=False, 
    drop_first=False, 
    dtype=None
)

### Save the lists  in one tensor

elecs_to_nr, # pandas df
muscles_to_nr # pandas df
raw_data_to_matrix, 
amplitudes_to_list, 
time_points_to_list

#### concatenate data to one df

In [224]:
##### concat muscles and elecs ###
muscles_to_nr.reset_index(drop=True, inplace=True)
elecs_to_nr.reset_index(drop=True, inplace=True)
muscles_to_nr.index = elecs_to_nr.index
muscles_and_elecs=pd.concat([muscles_to_nr, elecs_to_nr], axis=1)

##### add the raw data
raw_data_to_df = pd.DataFrame(raw_data_to_matrix) #convert to a pandas df
if len(raw_data_to_df.iloc[0])!=len((raw_data_to_matrix[0])): #get rid of nan column
    raw_data_to_df=raw_data_to_df.iloc[: , :-1]
raw_data_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs.reset_index(drop=True, inplace=True)
muscles_and_elecs.index = raw_data_to_df.index
muscles_and_elecs_and_raw=pd.concat([muscles_and_elecs, raw_data_to_df], axis=1)

##### add the amplitude
amplitudes_to_df=pd.DataFrame(amplitudes_to_list) #convert to a pandas df
amplitudes_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw.index = amplitudes_to_df.index
muscles_and_elecs_and_raw_and_amplitudes=pd.concat([muscles_and_elecs_and_raw, amplitudes_to_df], axis=1)

##### add time_points
time_points_to_df=pd.DataFrame(time_points_to_list) #convert to a pandas df
time_points_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw_and_amplitudes.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw_and_amplitudes.index = time_points_to_df.index
muscles_and_elecs_and_raw_and_amplitudes_and_time_points=pd.concat([muscles_and_elecs_and_raw_and_amplitudes, time_points_to_df], axis=1)
# print(pd.concat([muscles_and_elecs_and_raw_and_amplitudes[0:10], time_points_to_df[0:10]], axis=1))


#### convert df to tensor

In [225]:
#### Convert df to tensor ###
# determine the supported device
#with correction of nan values
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(muscles_and_elecs_and_raw_and_amplitudes_and_time_points):
    device = get_device()
    return torch.from_numpy(muscles_and_elecs_and_raw_and_amplitudes_and_time_points.values).float().to(device)

df_tensor = df_to_tensor(muscles_and_elecs_and_raw_and_amplitudes_and_time_points)
# series_tensor = df_to_tensor(series)
print(df_tensor[0:10])
print(df_tensor[0:10].shape)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03]])
torch.Size([10, 1440])


# Make a tensor with dimension (nElec, nMuscles, uniqeAmp, TPperTrial)

## Modify the data

In [226]:
device =  "cuda" if torch.cuda.is_available() else "cpu"

Remvove unecessary

In [227]:
#check which values are available in muscle and remove uncesessary ones, eg trig
print(pData["_0_0"].List_muscle_or_other) 
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
        pData[f'_{elec_i}_{muscle_i}'].List_muscle_or_other= ['LST', 'RST', 'LTA', 'LMG', 'LSol', 'LVLat', 'LRF', 'LIl', 'RSol', 'RMG', 'RTA', 'RVLat', 'RRF', 'RIl']    


['LST' 'RST' 'LTA' 'LMG' 'LSol' 'LVLat' 'LRF' 'LIl' 'RSol' 'RMG' 'RTA'
 'RVLat' 'RRF' 'RIl' 'Trig']


In [228]:
#remove uncesessary Elecs ones for now only the monopolar ones necessary
print(pData["_0_0"].List_elecs) 
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
        pData[f'_{elec_i}_{muscle_i}'].List_elecs=['Elec01_', 'Elec02_', 'Elec03_', 'Elec04_', 'Elec05_', 'Elec06_', 'Elec07_', 'Elec08_', 'Elec09_', 'Elec10_', 'Elec11_', 'Elec12_', 'Elec13_', 'Elec14_', 'Elec15_', 'Elec16_']

['Elec01_' 'Elec02_' 'Elec03_' 'Elec04_' 'Elec05_' 'Elec06_' 'Elec07_'
 'Elec08_' 'Elec09_' 'Elec10_' 'Elec11_' 'Elec12_' 'Elec13_' 'Elec14_'
 'Elec15_' 'Elec16_' 'Elec13_09101214' 'Elec14_10' 'Elec04_0305'
 'Elec02_03']


In [229]:
## n_Ampl_unique
## find all the amplitudes used(unique); to have one dim of amplitudes, where there is no trial for an amplitude, fill with nan

unique_amplitudes=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                unique_amplitudes.append(pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial])


# mylist = list(set(mylist))
unique_amplitudes=np.unique(unique_amplitudes)  
print(unique_amplitudes)          



[0.5 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3
 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7]


In [230]:
## n_duplicates_per_amp
## find the max nr of trials for a same amplitude; we add a dimension with nr of trials with the same amplitude, where there is no data, we fill with nan

trials_same_amp=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
        trials_same_amp.append(np.max(pd.Index(pData[f'_{elec_i}_{muscle_i}'].Amplitudes).value_counts())) #extract the max nr of repeated amp in one combination
       
max_trials_same_amp=np.max(trials_same_amp)



In [231]:
## raw_data_tp_per_trial
## find the max nr of raw_data_tp_per_trial; it can vary between 1403 and 1404, set 1404 as raw_data_tp_per_trial; where there is no data, we fill with nan
trial_nrs=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                trial_nrs.append(len(pData[f'_{elec_i}_{muscle_i}'].Raw[trial]))
       
pd.Index(trial_nrs).value_counts(sort=False) #outputs raw_data_tp_per_trial, and nr. of repeats for raw_data_tp_per_trial
raw_data_tp_per_trial=np.max(trial_nrs)

## Create tensor with raw_data

In [232]:
## newer version, with 1404 TP!
### Create tensor with size torch.Size([16, 14, 32, 11, 1404]) 
# nMuslces x nElecs x nAmp_unique x n_duplicates_per_amp x raw_data_tp_per_trial
raw_data_to_tensor=torch.ones(len(pData["_0_0"].List_elecs),len(pData["_0_0"].List_muscle_or_other),len(unique_amplitudes),max_trials_same_amp,raw_data_tp_per_trial)
raw_data_to_tensor.size()

torch.Size([16, 14, 32, 11, 1404])

In [233]:
## filling the 11 rows (repetition) for the same amp with the raw data/ resp nan value wheb there is no data for that amplitude
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            
        ind=0 # set index to zero, will serve as index in the amplitudes
        for pos_in_unique_amplitudes in range(len(unique_amplitudes)): #loop through all the amplitudes possible
            if len(pData[f'_{elec_i}_{muscle_i}'].Amplitudes.index) > ind:
                trial=pData[f'_{elec_i}_{muscle_i}'].Amplitudes.index[ind]
                print(f"Amplitude of unique amp:{unique_amplitudes[pos_in_unique_amplitudes]}, Trial_nr:{trial}")

                if pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial] > unique_amplitudes[pos_in_unique_amplitudes]:
                    print(f"Data for Amplitude {pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial]} too high for {unique_amplitudes[pos_in_unique_amplitudes]}, fill all the repeats with nan for this amplitude")
                    raw_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes, :, :]=torch.zeros((max_trials_same_amp,raw_data_tp_per_trial)).float()
                    raw_data_to_tensor[raw_data_to_tensor==0]= float('nan') #didn't find torch.nan function
                    
                elif pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial] == unique_amplitudes[pos_in_unique_amplitudes]:
                    print("same amplitude")
                    reps_for_amp=pd.Index(pData[f'_{elec_i}_{muscle_i}'].Amplitudes).value_counts(sort=False)[pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial]] #outputs the repeats of trials for the spec amplitude
                    
                    if reps_for_amp < max_trials_same_amp:
                    ##fill everything with nan values
                        raw_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes, :, :]=torch.zeros((max_trials_same_amp,raw_data_tp_per_trial)).float()
                        raw_data_to_tensor[raw_data_to_tensor==0]= float('nan') #didn't find torch.nan function

                        #save the trials with data
                    for repeats in range(reps_for_amp):
                        trial=pData[f'_{elec_i}_{muscle_i}'].Amplitudes.index[ind]
                        if len(pData[f'_{elec_i}_{muscle_i}'].Raw[trial]) < raw_data_tp_per_trial: #For trials with 1403 TP, only fill up 1403 and leave last position as nan
                            raw_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes,repeats,0:-1]=torch.Tensor(pData[f'_{elec_i}_{muscle_i}'].Raw[trial])
                            ind+=1
                        else:  #For trials with 1404 TP
                            raw_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes,repeats,:]=torch.Tensor(pData[f'_{elec_i}_{muscle_i}'].Raw[trial])
                            ind+=1

            else: #No more data for the remaining amplitudes in unique_amplitudes, thus fill with nan
                print(f"In combination {elec_i}_{muscle_i} no more data for {unique_amplitudes[pos_in_unique_amplitudes]}, fill all the repeats with nan for this and following amplitudes")
                raw_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes, :, :]=torch.zeros((max_trials_same_amp,raw_data_tp_per_trial)).float()
                raw_data_to_tensor[raw_data_to_tensor==0]= float('nan') #didn't find torch.nan function

Amplitude of unique amp:0.5, Trial_nr:27
Data for Amplitude 1.5 too high for 0.5, fill all the repeats with nan for this amplitude
Amplitude of unique amp:0.7, Trial_nr:27
Data for Amplitude 1.5 too high for 0.7, fill all the repeats with nan for this amplitude
Amplitude of unique amp:0.8, Trial_nr:27
Data for Amplitude 1.5 too high for 0.8, fill all the repeats with nan for this amplitude
Amplitude of unique amp:0.9, Trial_nr:27
Data for Amplitude 1.5 too high for 0.9, fill all the repeats with nan for this amplitude
Amplitude of unique amp:1.0, Trial_nr:27
Data for Amplitude 1.5 too high for 1.0, fill all the repeats with nan for this amplitude
Amplitude of unique amp:1.1, Trial_nr:27
Data for Amplitude 1.5 too high for 1.1, fill all the repeats with nan for this amplitude
Amplitude of unique amp:1.2, Trial_nr:27
Data for Amplitude 1.5 too high for 1.2, fill all the repeats with nan for this amplitude
Amplitude of unique amp:1.3, Trial_nr:27
Data for Amplitude 1.5 too high for 1.3, f

KeyboardInterrupt: 

Check 

In [ ]:
raw_data_to_tensor[15,13,8,:,:]
# torch.Size([15, 13, 32, 11, 1404])

tensor([[ 1.3417e-04, -1.1915e-03, -1.9731e-03,  ...,  1.0508e+00,
          9.0098e-01,         nan],
        [ 1.0869e-01,  1.0744e-01,  1.0264e-01,  ...,  1.0508e+00,
          9.0098e-01,         nan],
        [ 7.8775e-02,  7.8805e-02,  7.6033e-02,  ...,  1.0508e+00,
          9.0098e-01,         nan],
        ...,
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan]])

## Create tensor with amplitudes

In [ ]:

### Create tensor with size torch.Size([16, 14, 32]) 
# nMuslces x nElecs x nAmp_unique 
amplitude_data_to_tensor=torch.ones(len(pData["_0_0"].List_elecs),len(pData["_0_0"].List_muscle_or_other),len(unique_amplitudes))
amplitude_data_to_tensor.size()

torch.Size([16, 14, 32])

For every muscle and elec, save the amp where there is existing data, else save nan value.
Evt improve last else: to fill everything with nan and break innerloop, resp continue outer loop

In [ ]:
## newer version, taking repeats into account

## Starting with elec 0, muslce 0, amp=0.5, filling the 10rows (repetition) for the spec. amp with the raw data/ resp nan value since there is no data for that amplitude
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            
        ind=0 # set index to zero, will serve as index from amplitudes
        for pos_in_unique_amplitudes in range(len(unique_amplitudes)): #loop through all the amplitudes possible
            

            if len(pData[f'_{elec_i}_{muscle_i}'].Amplitudes.unique()) > ind: #The repeat of the amplitudes is not
                data_amp=pData[f'_{elec_i}_{muscle_i}'].Amplitudes.unique()[ind]
                print(f"Amplitude of unique amp:{unique_amplitudes[pos_in_unique_amplitudes]}, {elec_i}, {muscle_i},  data_amp:{data_amp}")

                if data_amp > unique_amplitudes[pos_in_unique_amplitudes]:
                    print(f"No data for Amplitude: {unique_amplitudes[pos_in_unique_amplitudes]}, next amp with data: {data_amp} fill  nan for this amplitude")
                    amplitude_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.zeros((1)).float()
                    amplitude_data_to_tensor[amplitude_data_to_tensor==0]= float('nan') #didn't find torch.nan function
                    
                elif data_amp == unique_amplitudes[pos_in_unique_amplitudes]:
                    #save the amp with data
                    print("same amplitude")
                
                    amplitude_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.Tensor([data_amp])
                    ind+=1
                    print("#################################################################")

            
            else: #No more data for the remaining amplitudes in unique_amplitudes, thus fill with nan
                print(f"In combination {elec_i}_{muscle_i} no more data for {unique_amplitudes[pos_in_unique_amplitudes]}, nan value for this and following amplitudes")
                # amplitude_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes:]=torch.zeros((len(unique_amplitudes)-pos_in_unique_amplitudes)).float()
                amplitude_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.zeros((1)).float()
                amplitude_data_to_tensor[amplitude_data_to_tensor==0]= float('nan') #didn't find torch.nan function


Amplitude of unique amp:0.5, 0, 0,  data_amp:1.5
No data for Amplitude: 0.5, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:0.7, 0, 0,  data_amp:1.5
No data for Amplitude: 0.7, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:0.8, 0, 0,  data_amp:1.5
No data for Amplitude: 0.8, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:0.9, 0, 0,  data_amp:1.5
No data for Amplitude: 0.9, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:1.0, 0, 0,  data_amp:1.5
No data for Amplitude: 1.0, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:1.1, 0, 0,  data_amp:1.5
No data for Amplitude: 1.1, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:1.2, 0, 0,  data_amp:1.5
No data for Amplitude: 1.2, next amp with data: 1.5 fill  nan for this amplitude
Amplitude of unique amp:1.3, 0, 0,  data_amp:1.5
No data for Amplitude: 1.3, next amp with

In [ ]:
amplitude_data_to_tensor[0,:,:] #torch.Size([16, 14, 32])

tensor([[   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
         1.5000,    nan, 1.7000,    nan, 1.9000,    nan, 2.1000,    nan, 2.3000,
            nan, 2.5000,    nan, 2.7000,    nan, 2.9000,    nan, 3.1000,    nan,
         3.3000,    nan, 3.5000,    nan, 3.7000],
        [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
         1.5000,    nan, 1.7000,    nan, 1.9000,    nan, 2.1000,    nan, 2.3000,
            nan, 2.5000,    nan, 2.7000,    nan, 2.9000,    nan, 3.1000,    nan,
         3.3000,    nan, 3.5000,    nan, 3.7000],
        [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
         1.5000,    nan, 1.7000,    nan, 1.9000,    nan, 2.1000,    nan, 2.3000,
            nan, 2.5000,    nan, 2.7000,    nan, 2.9000,    nan, 3.1000,    nan,
         3.3000,    nan, 3.5000,    nan, 3.7000],
        [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
         1.5000,    nan, 1.7000,    nan,

## Create tensor with Muscles

In [ ]:

### Create tensor with size torch.Size([16, 14]) 
# nMuslces x nElecs
muscles_to_tensor=torch.ones(len(pData["_0_0"].List_elecs),len(pData["_0_0"].List_muscle_or_other))
muscles_to_tensor.size()

torch.Size([16, 14])

In [ ]:


for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
        muscles_to_tensor[elec_i, muscle_i]=torch.Tensor([muscle_i])




In [ ]:
muscles_to_tensor

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  

## Create tensor with Elecs

In [ ]:

### Create tensor with size torch.Size([16]) 
# nMuslces x nElecs
elecs_to_tensor=torch.ones(len(pData["_0_0"].List_elecs))
elecs_to_tensor.size()
print(elecs_to_tensor)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [ ]:


for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    elecs_to_tensor[elec_i]=torch.Tensor([elec_i])
print(elecs_to_tensor)



tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15.])


## Reference table for the electrode and muscle naming

In [ ]:
df_electrodes=pd.DataFrame(pData["_0_0"].List_elecs, columns=["elec_name"])
# df_electrodes.iloc[0] # to acces elec name
df_muscles=pd.DataFrame(pData["_0_0"].List_muscle_or_other, columns=["muscle_name"])
# df_muscles.iloc[0] # to access muscle name

In [ ]:

nr=df_muscles.index.where(df_muscles.muscle_name=="LST").dropna().astype(int)[0]
print(nr)

0


# Save the data 

1 csv for df_electrodes (two columns, index, electrode_name)
1 csv for df_muscles (two columns, index, muscle_name)
1 file for Tensor for the raw data (EMG)
1 file for Tensor for the amplitudes

In [ ]:
df_electrodes.to_csv('df_electrodes.csv')
df_muscles.to_csv("df_muscles.csv")

## to retrieve ##
# df_electrodes = pd.read_csv("df_electrodes.csv")
# df_muscles= pd.read_csv("df_muscles.csv")

In [ ]:
import pickle
raw_data_to_tensor #torch.Size([16, 14, 32, 11, 1404])
amplitude_data_to_tensor #torch.Size([16, 14, 32])
muscles_to_tensor
elecs_to_tensor
file = open('/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/tensor_dump.pkl', "wb")
obj_1 = raw_data_to_tensor
obj_2 = amplitude_data_to_tensor 
obj_3 = muscles_to_tensor
obj_4 = elecs_to_tensor
pickle.dump(obj_1, file)
pickle.dump(obj_2, file)
pickle.dump(obj_3, file)
pickle.dump(obj_4, file)

file.close()


## to retrieve ##
# file = open('tensor_dump.pkl', 'rb')
# raw_data_to_tensor = pickle.load(file)
# amplitude_data_to_tensor  = pickle.load(file)
# muscles_to_tensor = pickle.load(file)
# elecs_to_tensor = pickle.load(file)
# print(obj_1)
# print(obj_2)
# print(obj_3)
# print(obj_4)
# file.close()